# Heisenberg Model Simulation using Trotterization
Pennylane challenge - https://pennylane.ai/challenges/heisenberg_model

In [75]:
import json
import pennylane as qml
import pennylane.numpy as np

from functools import partial

## Implementing without the use of built-in functions

In [105]:
num_wires = 4
dev = qml.device("default.mixed", wires=num_wires)


def noisy_IsingZZ(phi, p, wires):
    qml.CNOT(wires)
    qml.DepolarizingChannel(p, wires[1])
    qml.RZ(2*phi, wires[0])
    qml.CNOT(wires)
    qml.DepolarizingChannel(p, wires[1])

def CNOT_block(w1,w2,p):
        qml.CNOT(wires=[w1,w2])
        qml.DepolarizingChannel(p, wires=[w2])
        qml.RZ(-3.0, wires=w2)
        qml.CNOT(wires=[w1,w2])
        qml.DepolarizingChannel(p, wires=[w2])

def R_block(w):
    qml.RZ(1.57, wires=w)
    qml.RX(1.57, wires=w)
    qml.RZ(1.57, wires=w)


@qml.qnode(dev)
def heisenberg_trotter(couplings, p, time, depth):
    """This QNode returns the final state of the spin chain after evolution for a time t, 
    under the Trotter approximation of the exponential of the Heisenberg Hamiltonian.
    
    Args:
        couplings (list(float)): 
            An array of length 4 that contains the coupling constants and the magnetic field 
            strength, in the order [J_x, J_y, J_z, h].
        p (float): The depolarization probability after each CNOT gate.
        depth (int): The Trotterization depth.
        time (float): Time during which the state evolves

    Returns:
        (numpy.tensor): The evolved quantum state.
    """

    # First layer of single-qubit gates
    qml.RZ(1.57, wires=0)
    qml.RX(1.57, wires=0)
    qml.RZ(1.57, wires=0)
    qml.RZ(1.57, wires=3)
    qml.RX(1.57, wires=3)
    qml.RZ(1.57, wires=3)

    # First block of CNOT + RZ(-3.0)
    CNOT_block(0, 3, p)

    # Middle layers (manually translated)
    for i in range(num_wires):
        R_block(i)

    qml.RZ(1.57, wires=0)
    qml.RX(1.57, wires=0)
    qml.RZ(1.57, wires=0)

    qml.RZ(1.57, wires=3)
    qml.RX(1.57, wires=3)
    qml.RZ(1.57, wires=3)

    CNOT_block(3, 2, p)
    R_block(2)
    R_block(3)
    R_block(2)
    qml.RX(1.57, wires=3)

    CNOT_block(2, 1, p)
    R_block(1)
    R_block(2)
    R_block(1)

    CNOT_block(1, 0, p)
    R_block(0)
    R_block(1)
    qml.RX(1.57, wires=0)

    CNOT_block(0, 3, p)

    # More RX layers
    qml.RX(-1.57, wires=0)
    qml.RX(1.57, wires=0)
    qml.RX(1.57, wires=1)
    qml.RX(1.57, wires=2)
    qml.RX(-1.57, wires=3)
    qml.RX(1.57, wires=3)

    CNOT_block(3, 2, p)
    qml.RX(-1.57, wires=2)
    qml.RX(1.57, wires=2)
    qml.RX(-1.57, wires=3)

    CNOT_block(2, 1, p)    
    qml.RX(-1.57, wires=1)
    qml.RX(1.57, wires=1)
    qml.RX(-1.57, wires=2)

    CNOT_block(1, 0, p)
    qml.RX(-1.57, wires=0)
    qml.RX(-1.57, wires=1)

    CNOT_block(0, 3, p)
    CNOT_block(3, 2, p)
    R_block(3)
    CNOT_block(2, 1, p)
    R_block(2)
    qml.RZ(-5.40, wires=3)
    qml.RZ(-5.40, wires=2)
    CNOT_block(1, 0, p)

    # Middle layers (manually translated)
    for i in range(num_wires):
        R_block(i)
    qml.RZ(-5.40, wires=0)
    qml.RZ(-5.40, wires=1)
    R_block(0)
    R_block(1)
    
    return qml.state()

In [106]:
print(qml.draw(heisenberg_trotter)([0.5,0.5,0.5,0.9], 0, 3, 1))

0: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╭●───────────────────────────────────────╭●
1: ───────────────────────────────│────────────────────────────────────────│─
2: ───────────────────────────────│────────────────────────────────────────│─
3: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╰X──DepolarizingChannel(0.00)──RZ(-3.00)─╰X

───RZ(1.57)───────────────────RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)──RZ(1.57)─────────────
───RZ(1.57)───────────────────RX(1.57)──RZ(1.57)───────────────────────────────────────────
───RZ(1.57)───────────────────RX(1.57)──RZ(1.57)─────────────────────────────────────────╭X
───DepolarizingChannel(0.00)──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)──RZ(1.57)─╰●

───────────────────────────────────────────────────────────────────────────────────────────────────
───────────────────────────────────────────────────────────────────────────────────────────────────
───DepolarizingChannel(0.00)──RZ(-3.00)─╭X──DepolarizingChannel(0.00)──RZ(1.57)──RX(1.57)──RZ(1.57)
──────────────────

In [86]:
def calculate_fidelity(couplings, p, time, depth):
    """This function returns the fidelity between the final states of the noisy and
    noiseless Trotterizations of the Heisenberg models, using only CNOT and rotation gates

    Args:
        couplings (list(float)): 
            A list with the J_x, J_y, J_z and h parameters in the Heisenberg Hamiltonian, as
            defined in the problem statement.
        p (float): The depolarization probability of the depolarization gate that acts on the
                   target qubit of each CNOT gate.
        time (float): The period of time evolution simulated by the Trotterization.
        depth (int): The Trotterization depth.

    Returns:
        (float): Fidelity between final states of the noisy and noiseless Trotterizations
    """
    return qml.math.fidelity(heisenberg_trotter(couplings,0,time, depth),heisenberg_trotter(couplings,p,time,depth))

In [107]:
print(calculate_fidelity([1,3,2,0.3],0.0,2.5,2))

1.00000006094097


## Implementing with built-in functions

In [94]:
def create_hamiltonian(params):
    couplings = [-params[-1]]
    ops = [qml.PauliX(3)]
    for i in range(3):
        couplings = [-params[-1]] + couplings
        ops = [qml.PauliX(i)] + ops        
    for i in range(4):
        couplings = [-params[-2]] + couplings
        ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops
    for i in range(4):
        couplings = [-params[-3]] + couplings
        ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops
    for i in range(4):
        couplings = [-params[0]] + couplings
        ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    
    return qml.Hamiltonian(couplings,ops)

@partial(qml.transforms.decompose, gate_set={qml.CNOT, qml.RX, qml.RY, qml.RZ})
@qml.qnode(dev)
def evolve(params, time, depth):
    qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)
    return qml.state()

In [69]:
print(create_hamiltonian([1,3,2,0.3]))

-1 * (X(3) @ X(0)) + -1 * (X(2) @ X(3)) + -1 * (X(1) @ X(2)) + -1 * (X(0) @ X(1)) + -3 * (Y(3) @ Y(0)) + -3 * (Y(2) @ Y(3)) + -3 * (Y(1) @ Y(2)) + -3 * (Y(0) @ Y(1)) + -2 * (Z(3) @ Z(0)) + -2 * (Z(2) @ Z(3)) + -2 * (Z(1) @ Z(2)) + -2 * (Z(0) @ Z(1)) + -0.3 * X(2) + -0.3 * X(1) + -0.3 * X(0) + -0.3 * X(3)


In [104]:
print(qml.draw(evolve)([0.5,0.5,0.5,0.9], 3, 1))

0: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╭●────────────╭●──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)
1: ───────────────────────────────│─────────────│───RZ(1.57)──RX(1.57)──RZ(1.57)────────────────────
2: ───────────────────────────────│─────────────│───RZ(1.57)──RX(1.57)──RZ(1.57)────────────────────
3: ──RZ(1.57)──RX(1.57)──RZ(1.57)─╰X──RZ(-3.00)─╰X──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)

───RZ(1.57)────────────────────────────────────────────────────────────────────────────────
─────────────────────────────────────────────────────────────────────────────────────────╭X
────────────╭X──RZ(-3.00)─╭X──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)──RZ(1.57)─╰●
───RZ(1.57)─╰●────────────╰●──RZ(1.57)──RX(1.57)──RZ(1.57)──RX(1.57)───────────────────────

────────────────────────────────────────────────────────────────────────────╭X──RZ(-3.00)─╭X
───RZ(-3.00)─╭X──RZ(1.57)──RX(1.57)──RZ(1.57)──RZ(1.57)──RX(1.57)──RZ(1.57)─╰●────────────╰●
─────────────╰●──RZ(1.57)──RX(1.57)──RZ(

## Testing by computing Fidelity

In [109]:
random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )
x = qml.math.fidelity(heisenberg_trotter([1,3,2,0.3],0.0,2.5,2),evolve([1,3,2,0.3],2.5,2))
print(x)

0.09053263086987298


In [110]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output =calculate_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises: 
            ``AssertionError`` if the solution output is incorrect in any way.
            
    """
    def create_hamiltonian(params):

        couplings = [-params[-1]]
        ops = [qml.PauliX(3)]

        for i in range(3):

            couplings = [-params[-1]] + couplings
            ops = [qml.PauliX(i)] + ops        

        for i in range(4):

            couplings = [-params[-2]] + couplings
            ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[-3]] + couplings
            ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[0]] + couplings
            ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    

        return qml.Hamiltonian(couplings,ops)

    @qml.qnode(dev)
    def evolve(params, time, depth):

        qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)

        return qml.state()
    
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    tape = heisenberg_trotter.qtape
    names = [op.name for op in tape.operations]
    
    random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )

    fidelity = qml.math.fidelity(heisenberg_trotter(random_params,0,1,2),evolve(random_params,1,2))
    print(f"Fidelity: {fidelity}")
    
    assert fidelity >= 1, "Your circuit does not Trotterize the Heisenberg Model"
    
    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization"
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must only use RX, RY, RZ, CNOT, and depolarizing gates (don't use qml.Rot or Paulis)"
    
    assert solution_output >= expected_output-0.005, "Your fidelity is not high enough. You may be using more CNOT gates than needed"

# These are the public test cases
test_cases = [
    ('[[1,2,1,0.3],0.05,2.5,1]', '0.33723981123369573'),
    ('[[1,3,2,0.3],0.05,2.5,2]', '0.15411351752086694')
]
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,2,1,0.3],0.05,2.5,1]'...
Fidelity: 0.3157651138704102


c:\Users\pranv\pennylane\.venv\Lib\site-packages\pennylane\workflow\qnode.py:830: PennyLaneDeprecationWarning: The tape/qtape property is deprecated and will be removed in v0.41. Instead, use the qml.workflow.construct_tape function.
  warnings.warn(


AssertionError: Your circuit does not Trotterize the Heisenberg Model